In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import AutoModel
import torch
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\AJDAR\Desktop\dat_science_sources\pytorch_deep_learning_book\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [158]:
from datasets import Dataset

def load_conllpp_no_id(path):
    tokens_list, pos_list, chunk_list, ner_list = [], [], [], []
    words, pos, chunk, ner = [], [], [], []

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
          

          # if line is not empty
            line = line.strip()
            if not line:  
                if words:
                    tokens_list.append(words)
                    pos_list.append(pos)
                    chunk_list.append(chunk)
                    ner_list.append(ner)
                    words, pos, chunk, ner = [], [], [], []
                continue


                # skip broken lines
            splits = line.split()  
            if len(splits) < 4:
                continue  
            word, pos_tag, chunk_tag, ner_tag = splits[:4]
            words.append(word)
            pos.append(pos_tag)
            chunk.append(chunk_tag)
            ner.append(ner_tag)

    # catch last sentence if file doesn't end with blank line
    if words:
        tokens_list.append(words)
        pos_list.append(pos)
        chunk_list.append(chunk)
        ner_list.append(ner)

    return Dataset.from_dict({
        "tokens": tokens_list,
        "pos_tags": pos_list,
        "chunk_tags": chunk_list,
        "ner_tags": ner_list
    })

In [159]:
# Example usage:
train_dataset = load_conllpp_no_id("../data/conllpp_train.txt")
test_dataset = load_conllpp_no_id("../data/conllpp_test.txt")
eval_dataset = load_conllpp_no_id("../data/conllpp_dev.txt")

print(test_dataset[3])

{'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'], 'pos_tags': ['NNP', ',', 'NNP', 'NNP', 'NNPS', 'CD'], 'chunk_tags': ['I-NP', 'O', 'I-NP', 'I-NP', 'I-NP', 'I-NP'], 'ner_tags': ['B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O']}


In [161]:
#adding id to Dataset

if "id" not in train_dataset.features.keys():
    train_dataset = train_dataset.add_column("id", list(range(len(train_dataset))))
if "id" not in test_dataset.features.keys():
    test_dataset = test_dataset.add_column("id", list(range(len(test_dataset))))
if "id" not in eval_dataset.features.keys():
    eval_dataset = eval_dataset.add_column("id", list(range(len(eval_dataset))))

# print(eval_dataset.features)

In [163]:
from datasets import DatasetDict

dataset = DatasetDict({
    "train": train_dataset,
    "validation": eval_dataset,
    "test": test_dataset
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id'],
        num_rows: 14987
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id'],
        num_rows: 3466
    })
    test: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id'],
        num_rows: 3684
    })
})


In [164]:
df_train = pd.DataFrame(dataset["train"][:])[["tokens","ner_tags"]]
df_train.head()

,tokens,ner_tags
0,[-DOCSTART-],[O]
1,"[EU, rejects, German, call, to, boycott, Briti...","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]"
2,"[Peter, Blackburn]","[B-PER, I-PER]"
3,"[BRUSSELS, 1996-08-22]","[B-LOC, O]"
4,"[The, European, Commission, said, on, Thursday...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,..."


In [110]:
dataset["train"].features

{'tokens': List(Value('string')),
 'pos_tags': List(Value('string')),
 'chunk_tags': List(Value('string')),
 'ner_tags': List(Value('string')),
 'id': Value('int64')}

In [165]:
tags = dataset["train"]["ner_tags"]
list(tags)

[['O'],
 ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O'],
 ['O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I

In [166]:
all_tags =set([tag for element in list(tags) for tag in element])
all_tags

{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}

In [167]:
id2tag = {id:tag for id,tag in enumerate(all_tags)}

tag2id  = {tag:id for id,tag in enumerate(all_tags)}
tag2id

{'I-PER': 0,
 'I-ORG': 1,
 'B-PER': 2,
 'I-MISC': 3,
 'B-ORG': 4,
 'B-MISC': 5,
 'I-LOC': 6,
 'O': 7,
 'B-LOC': 8}

In [175]:
# ex = ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

def create_tag_numbers(batch):
    ner_tags_number = {"ner_tags_number" : [tag2id[element] for element in batch["ner_tags"] ]}

    return ner_tags_number
   

In [180]:
dataset_test = dataset
dataset= dataset.map(create_tag_numbers)


Map: 100%|██████████| 3684/3684 [00:00<00:00, 16944.64 examples/s]


In [181]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id', 'ner_tags_number'],
        num_rows: 14987
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id', 'ner_tags_number'],
        num_rows: 3466
    })
    test: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id', 'ner_tags_number'],
        num_rows: 3684
    })
})

In [182]:
model_ckp = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckp)

In [212]:
inputs = dataset["train"][5]["tokens"]
inputs = tokenizer(inputs, is_split_into_words=True)
inputs.tokens()
# inputs.word_ids()

['[CLS]',
 'germany',
 "'",
 's',
 'representative',
 'to',
 'the',
 'european',
 'union',
 "'",
 's',
 'veterinary',
 'committee',
 'werner',
 'z',
 '##wing',
 '##mann',
 'said',
 'on',
 'wednesday',
 'consumers',
 'should',
 'buy',
 'sheep',
 '##me',
 '##at',
 'from',
 'countries',
 'other',
 'than',
 'britain',
 'until',
 'the',
 'scientific',
 'advice',
 'was',
 'clearer',
 '.',
 '[SEP]']

In [206]:
def align_labels_with_tokens(labels,word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id!=current_word:
            current_word = word_id 
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)

        elif word_id is None:
            new_labels.append(-100)

        else:
            label = labels[word_id]

            if label%2 == 1:
                label +=1
            new_labels.append(label)

    return new_labels



In [213]:
labels = dataset["train"][5]["ner_tags_number"]
word_ids = inputs.word_ids()
print(labels, word_ids)

[8, 7, 7, 7, 7, 4, 1, 7, 7, 7, 2, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7] [None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [220]:
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation = True, is_split_into_words=True)
    all_labels = examples["ner_tags_number"]
    
    new_labels = []
    for i , labels in enumerate(all_labels):
       word_ids = tokenized_inputs.word_ids(i)
       new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels

    return tokenized_inputs
    

In [221]:
tokenized_datasets = dataset.map(tokenize_and_align , batched = True, remove_columns=dataset["train"].column_names)

Map: 100%|██████████| 3684/3684 [00:00<00:00, 21758.92 examples/s]


In [224]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14987
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3466
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3684
    })
})

In [225]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [226]:
batch = data_collator([tokenized_datasets["train"][i] for i in range (2)])

In [227]:
batch

{'input_ids': tensor([[  101,  1011,  9986, 14117,  2102,  1011,   102,     0,     0,     0,
             0],
        [  101,  7327, 19164,  2446,  2655,  2000, 17757,  2329, 12559,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100,    7,    8,    8,    8,    8, -100, -100, -100, -100, -100],
        [-100,    4,    7,    5,    7,    7,    7,    5,    7,    7, -100]])}

In [228]:
import evaluate
metric = evaluate.load("seqeval")

In [231]:
ner_feature = dataset['train'].features
ner_feature

{'tokens': List(Value('string')),
 'pos_tags': List(Value('string')),
 'chunk_tags': List(Value('string')),
 'ner_tags': List(Value('string')),
 'id': Value('int64'),
 'ner_tags_number': List(Value('int64'))}

In [238]:
label_names = all_tags
label_names

{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}

In [237]:
labels = dataset['train'][1]['ner_tags']
# labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [239]:
predictions = labels.copy()
predictions[2] = "O"

metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [240]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [241]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}

In [242]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
                                                    model_ckp,
                                                    id2label=id2label,
                                                    label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [246]:
from transformers import TrainingArguments

args = TrainingArguments(output_dir ="distilbert-finetuned-ner",
                        #  evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=3,
                         weight_decay=0.01)

In [247]:
from transformers import Trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

C:\Users\AJDAR\AppData\Local\Temp\ipykernel_22172\2906752468.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


Step,Training Loss
500,0.342700
1000,0.115400
1500,0.103700
2000,0.075800
2500,0.052500
3000,0.052200
3500,0.045800
4000,0.038800
4500,0.027200
5000,0.027300


TrainOutput(global_step=5622, training_loss=0.0814727843761953, metrics={'train_runtime': 192.5541, 'train_samples_per_second': 233.498, 'train_steps_per_second': 29.197, 'total_flos': 468432016607898.0, 'train_loss': 0.0814727843761953, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

checkpoint = "./distilbert-finetuned-ner/checkpoint-5622"
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

token_classifier("My name is Laxmi Kant Tiwari. I work at KGP Talkie and live in Mumbai")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.99875784,
  'word': 'lax',
  'start': 11,
  'end': 14},
 {'entity_group': 'PER',
  'score': 0.9979857,
  'word': '##mi kant tiwari',
  'start': 14,
  'end': 28},
 {'entity_group': 'ORG',
  'score': 0.9971048,
  'word': 'kg',
  'start': 40,
  'end': 42},
 {'entity_group': 'ORG',
  'score': 0.984998,
  'word': '##p talk',
  'start': 42,
  'end': 48},
 {'entity_group': 'PER',
  'score': 0.9747114,
  'word': '##ie',
  'start': 48,
  'end': 50},
 {'entity_group': 'LOC',
  'score': 0.9985481,
  'word': 'mumbai',
  'start': 63,
  'end': 69}]

: 